In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow.keras as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
#from keras.applications.inception_v3 import InceptionV3

from keras.applications import MobileNetV2, VGG19, VGG16, Xception, InceptionV3, InceptionResNetV2, DenseNet201
import cv2
import os

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/face-mask-detection-dataset/submission.csv
/kaggle/input/face-mask-detection-dataset/train.csv
/kaggle/input/face-mask-detection-dataset/Medical mask/Medical mask/meta.json


In [ ]:
data = pd.read_csv('/kaggle/input/face-mask-detection-dataset/train.csv')
deneme = [1 if ((element=='face_with_mask') or  (element=='mask_surgical') or (element=='mask_colorful')) else 0 for element in data.classname]
data['Classification'] = deneme
data.drop(['x1', 'x2', 'y1', 'y2', 'classname'], axis =1, inplace=True)
data

In [ ]:
datagrouped = data.groupby(['name'], as_index=False).sum()
datagrouped.Classification[datagrouped['Classification']>=1]=1
datagrouped

In [ ]:
path = '/kaggle/input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/images/'
rre = cv2.imread(path+'3216.png')
plt.imshow(rre)

In [ ]:
path = '/kaggle/input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/images/'
trainingdata = np.zeros((1,256,256,3))
k = 0
for element in datagrouped['name']:
    resim = cv2.imread(path+element)
    resim = cv2.resize(resim,(256,256), interpolation = cv2.INTER_NEAREST)
    resim = (resim-np.mean(resim))/np.std(resim)
    trainingdata = np.append(trainingdata, resim[np.newaxis,...], axis=0) 
    if k==1000:
        break
    k+=1

In [ ]:
ag = np.delete(trainingdata,0, axis=0)
ag.shape

In [ ]:
plt.imshow(ag[25,...])

In [ ]:
sns.distplot(ag[25,125:225,50:150,0])

In [ ]:
sns.distplot(ag[25,...,0])

In [ ]:
plt.imshow(ag[25,...]*(ag[25,...]>0.6))

In [ ]:
trainingdata[1000:1500,...].shape

In [ ]:
#labeldata = np.zeros((1000,2))
#for el, em in enumerate(labeldata):
#    if em == 0:
#        continue
#    else:        
labeldata = datagrouped.Classification[:1001]
labeldata

In [ ]:
input_layer = tf.Input(shape=(256, 256, 3))
#model = VGG19(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
#for layer in model.layers[:-3]:
#    layer.trainable = False
x = tf.layers.Conv2D(128,(3, 3), strides=(1, 1), activation='relu')(input_layer)
x = tf.layers.Conv2D(128,(3, 3), strides=(1, 1), activation='relu')(x)
#x = tf.layers.Conv2D(64,(3, 3), strides=(1, 1), activation='relu')(x)
#x = tf.layers.Conv2D(128,(3, 3), padding='same', strides=(1, 1), activation='relu')(x)
x = tf.layers.BatchNormalization()(x)
x = tf.layers.MaxPool2D((2, 2), strides=(2, 2))(x)
x = tf.layers.Dropout(0.5)(x)
x = tf.layers.Conv2D(64,(3, 3), strides=(1, 1), activation='relu')(x)
x = tf.layers.Conv2D(64,(3, 3), strides=(1, 1), activation='relu')(x)
#x = tf.layers.Conv2D(64,(3, 3), strides=(1, 1), activation='relu')(x)
x = tf.layers.BatchNormalization()(x)
#x = tf.layers.Conv2D(64,(3, 3), strides=(1, 1), activation='relu')(x)
#x = tf.layers.Conv2D(256,(3, 3), padding='same', strides=(1, 1), activation='relu')(x)
x = tf.layers.MaxPool2D((2, 2), strides=(2, 2))(x)
x = tf.layers.Conv2D(32,(3, 3), strides=(1, 1), activation='relu')(x)
x = tf.layers.Conv2D(32,(3, 3), strides=(1, 1), activation='relu')(x)
x = tf.layers.BatchNormalization()(x)
#x = tf.layers.Conv2D(64,(3, 3), strides=(1, 1), activation='relu')(x)
#x = tf.layers.Conv2D(512,(3, 3), padding='same', strides=(1, 1), activation='relu')(x)
x = tf.layers.MaxPool2D((2, 2), strides=(2, 2))(x)
#x = tf.layers.Conv2D(64,(3, 3), strides=(1, 1), activation='relu')(x)
x = tf.layers.Dropout(0.5)(x)
#x = tf.layers.Conv2D(64,(3, 3), strides=(1, 1), activation='relu')(x)
#x = tf.layers.Conv2D(64,(3, 3), strides=(1, 1), activation='relu')(x)
#x = tf.layers.BatchNormalization()(x)
#x = tf.layers.Conv2D(32,(3, 3), strides=(1, 1), activation='relu')(x)
#x = tf.layers.Conv2D(32,(3, 3), strides=(1, 1), activation='relu')(x)
#x = tf.layers.Conv2D(32,(3, 3), strides=(1, 1), activation='relu')(x)
#x = tf.layers.Conv2D(32,(3, 3), strides=(1, 1), activation='relu')(x)
#x = tf.layers.Conv2D(256,(3, 3), padding='same', strides=(1, 1), activation='relu')(x)
#x = tf.layers.MaxPool2D((2, 2), padding='same', strides=(1, 1))(x)
#x = tf.layers.BatchNormalization()(x)
#x = tf.layers.Conv2D(256, (1, 1), padding='same', strides=(1, 1), activation='relu')(x)
#x = tf.layers.Conv2D(256, (1, 1), padding='same', strides=(1, 1), activation='relu')(x)
#x = tf.layers.MaxPool2D((2, 2), padding='same', strides=(1, 1))(x)
#x = tf.layers.Conv2D(256, (1, 1), padding='same', strides=(1, 1), activation='relu')(x)
#x = tf.layers.Conv2D(256, (1, 1), padding='same', strides=(1, 1), activation='relu')(x)
#x = tf.layers.MaxPool2D((2, 2), padding='same', strides=(1, 1))(x)
#x = tf.layers.Conv2D(256, (1, 1), padding='same', strides=(1, 1), activation='relu')(x)
#x = tf.layers.Conv2D(256, (1, 1), padding='same', strides=(1, 1), activation='relu')(x)
#x = tf.layers.MaxPool2D((2, 2), strides=(2, 2))(x)
#x = tf.layers.Dropout(0.5)(x)
x = tf.layers.Conv2D(16, (1, 1), padding='same', strides=(1, 1), activation='relu')(x)
x = tf.layers.Conv2D(16, (1, 1), strides=(2, 2), activation='relu')(x)
x = tf.layers.BatchNormalization()(x)
#x = tf.layers.Conv2D(1, (3, 3), strides=(2, 2), activation='relu')(x)
#x = tf.layers.MaxPool2D((2, 2), padding='same', strides=(1, 1))(x)
#x = tf.layers.GlobalMaxPooling2D()(x)
#x = tf.layers.BatchNormalization()(x)
#x = tf.layers.Lambda(lambda x: x**2)(x)
x = tf.layers.Flatten()(x)
#x = tf.layers.GlobalAveragePooling2D()(x)
#x = tf.layers.Dense(1024, activation='relu')(x)
#x = tf.layers.Dense(2048, activation='relu')(x)
#x = tf.layers.Dense(1024, activation='relu')(x)
x = tf.layers.Dense(128, activation='relu')(x)
x = tf.layers.Dense(256, activation='relu')(x)
x = tf.layers.Dense(128, activation='relu')(x)
x = tf.layers.Dense(1, activation='sigmoid')(x)
cmodel = tf.Model(input_layer, x)
cmodel.summary()

In [ ]:
model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
for layer in model.layers[:-8]:
    layer.trainable = False
x = tf.layers.GlobalAveragePooling2D()(model.output)
#x = tf.layers.Flatten()(model.output)
#x = tf.layers.BatchNormalization()(x)
#x = tf.layers.Dropout(0.5)(x)
#x = tf.layers.Dense(2048, activation='relu')(x)
x = tf.layers.Dense(8, activation=tf.layers.PReLU())(x)
x = tf.layers.BatchNormalization()(x)
x = tf.layers.Dropout(0.5)(x)
x = tf.layers.Dense(32, activation=tf.layers.PReLU())(x)
x = tf.layers.BatchNormalization()(x)
x = tf.layers.Dropout(0.5)(x)
x = tf.layers.Dense(64, activation=tf.layers.PReLU())(x)
x = tf.layers.BatchNormalization()(x)
x = tf.layers.Dropout(0.5)(x)
x = tf.layers.Dense(128, activation=tf.layers.PReLU())(x)
x = tf.layers.BatchNormalization()(x)
x = tf.layers.Dropout(0.5)(x)
x = tf.layers.Dense(256, activation=tf.layers.PReLU())(x)
x = tf.layers.BatchNormalization()(x)
x = tf.layers.Dropout(0.8)(x)
x = tf.layers.Dense(512, activation=tf.layers.PReLU())(x)
x = tf.layers.BatchNormalization()(x)
x = tf.layers.Dropout(0.8)(x)
x = tf.layers.Dense(1024, activation=tf.layers.PReLU())(x)
x = tf.layers.BatchNormalization()(x)
x = tf.layers.Dropout(0.8)(x)
x = tf.layers.Dense(1024, activation=tf.layers.PReLU())(x)
x = tf.layers.BatchNormalization()(x)
x = tf.layers.Dropout(0.8)(x)
x = tf.layers.Dense(1024, activation=tf.layers.PReLU())(x)
x = tf.layers.BatchNormalization()(x)
x = tf.layers.Dropout(0.8)(x)
x = tf.layers.Dense(512, activation=tf.layers.PReLU())(x)
x = tf.layers.BatchNormalization()(x)
x = tf.layers.Dropout(0.8)(x)
x = tf.layers.Dense(256, activation=tf.layers.PReLU())(x)
x = tf.layers.BatchNormalization()(x)
x = tf.layers.Dropout(0.8)(x)
x = tf.layers.Dense(128, activation=tf.layers.PReLU())(x)
x = tf.layers.BatchNormalization()(x)
x = tf.layers.Dense(64, activation=tf.layers.PReLU())(x)
x = tf.layers.BatchNormalization()(x)
x = tf.layers.Dense(32, activation=tf.layers.PReLU())(x)
x = tf.layers.BatchNormalization()(x)
x = tf.layers.Dense(16, activation=tf.layers.PReLU())(x)
x = tf.layers.BatchNormalization()(x)
x = tf.layers.Dense(8, activation=tf.layers.PReLU())(x)
x = tf.layers.BatchNormalization()(x)
x = tf.layers.Dropout(0.5)(x)
#x = tf.layers.Dropout(0.8)(x)
#x = tf.layers.BatchNormalization()(x)
#x = tf.layers.Dropout(0.5)(x)
#x = tf.layers.Dense(1024, activation='relu')(x)
#x = tf.layers.BatchNormalization()(x)
#x = tf.layers.Dropout(0.5)(x)
#x = tf.layers.ActivityRegularization()(x)
x = tf.layers.Dense(1, activation='sigmoid')(x)
m = tf.Model(inputs=model.input, outputs=x)

m.summary()

In [16]:
m.compile(loss=tf.losses.BinaryCrossentropy(), optimizer='nadam', metrics=["accuracy"])
m.fit(ag[0:800,...], labeldata[0:800], validation_data = (ag[800:1000,...],labeldata[800:1000]), epochs=300, batch_size=100)

Epoch 1/200
8/8 [==============================] - 31s 4s/step - loss: 0.9617 - accuracy: 0.4700 - val_loss: 0.6920 - val_accuracy: 0.8050
Epoch 2/200
8/8 [==============================] - 30s 4s/step - loss: 0.8931 - accuracy: 0.4900 - val_loss: 0.6909 - val_accuracy: 0.8200
Epoch 3/200
8/8 [==============================] - 31s 4s/step - loss: 0.8981 - accuracy: 0.4900 - val_loss: 0.6788 - val_accuracy: 0.8200
Epoch 4/200
8/8 [==============================] - 35s 4s/step - loss: 0.8873 - accuracy: 0.4837 - val_loss: 0.6773 - val_accuracy: 0.8200
Epoch 5/200
8/8 [==============================] - 31s 4s/step - loss: 0.8567 - accuracy: 0.4850 - val_loss: 0.6685 - val_accuracy: 0.8200
Epoch 6/200
8/8 [==============================] - 30s 4s/step - loss: 0.8493 - accuracy: 0.4812 - val_loss: 0.6785 - val_accuracy: 0.8200
Epoch 7/200
8/8 [==============================] - 30s 4s/step - loss: 0.8249 - accuracy: 0.4988 - val_loss: 0.6640 - val_accuracy: 0.8200
Epoch 8/200
8/8 [==========

8/8 [==============================] - 30s 4s/step - loss: 0.6159 - accuracy: 0.7150 - val_loss: 0.5206 - val_accuracy: 0.8200
Epoch 60/200
8/8 [==============================] - 30s 4s/step - loss: 0.6146 - accuracy: 0.7150 - val_loss: 0.5186 - val_accuracy: 0.8200
Epoch 61/200
8/8 [==============================] - 30s 4s/step - loss: 0.6356 - accuracy: 0.7100 - val_loss: 0.5241 - val_accuracy: 0.8200
Epoch 62/200
8/8 [==============================] - 32s 4s/step - loss: 0.6124 - accuracy: 0.7150 - val_loss: 0.5199 - val_accuracy: 0.8200
Epoch 63/200
8/8 [==============================] - 31s 4s/step - loss: 0.6096 - accuracy: 0.7237 - val_loss: 0.5201 - val_accuracy: 0.8200
Epoch 64/200
8/8 [==============================] - 29s 4s/step - loss: 0.6145 - accuracy: 0.7237 - val_loss: 0.5193 - val_accuracy: 0.8200
Epoch 65/200
8/8 [==============================] - 30s 4s/step - loss: 0.6240 - accuracy: 0.7088 - val_loss: 0.5184 - val_accuracy: 0.8200
Epoch 66/200
8/8 [===============

8/8 [==============================] - 31s 4s/step - loss: 0.5947 - accuracy: 0.7287 - val_loss: 0.5095 - val_accuracy: 0.8200
Epoch 118/200
8/8 [==============================] - 30s 4s/step - loss: 0.5901 - accuracy: 0.7337 - val_loss: 0.5088 - val_accuracy: 0.8200
Epoch 119/200
8/8 [==============================] - 31s 4s/step - loss: 0.5935 - accuracy: 0.7287 - val_loss: 0.5086 - val_accuracy: 0.8200
Epoch 120/200
8/8 [==============================] - 30s 4s/step - loss: 0.5847 - accuracy: 0.7375 - val_loss: 0.5080 - val_accuracy: 0.8200
Epoch 121/200
8/8 [==============================] - 30s 4s/step - loss: 0.5975 - accuracy: 0.7275 - val_loss: 0.5083 - val_accuracy: 0.8200
Epoch 122/200
8/8 [==============================] - 29s 4s/step - loss: 0.5829 - accuracy: 0.7350 - val_loss: 0.5066 - val_accuracy: 0.8200
Epoch 123/200
8/8 [==============================] - 30s 4s/step - loss: 0.5931 - accuracy: 0.7287 - val_loss: 0.5066 - val_accuracy: 0.8200
Epoch 124/200
8/8 [========

8/8 [==============================] - 30s 4s/step - loss: 0.5912 - accuracy: 0.7337 - val_loss: 0.4974 - val_accuracy: 0.8200
Epoch 176/200
8/8 [==============================] - 30s 4s/step - loss: 0.5826 - accuracy: 0.7325 - val_loss: 0.4964 - val_accuracy: 0.8200
Epoch 177/200
8/8 [==============================] - 30s 4s/step - loss: 0.5787 - accuracy: 0.7387 - val_loss: 0.4967 - val_accuracy: 0.8200
Epoch 178/200
8/8 [==============================] - 30s 4s/step - loss: 0.5805 - accuracy: 0.7350 - val_loss: 0.4970 - val_accuracy: 0.8200
Epoch 179/200
8/8 [==============================] - 30s 4s/step - loss: 0.5860 - accuracy: 0.7375 - val_loss: 0.4972 - val_accuracy: 0.8200
Epoch 180/200
8/8 [==============================] - 29s 4s/step - loss: 0.5841 - accuracy: 0.7337 - val_loss: 0.4975 - val_accuracy: 0.8200
Epoch 181/200
8/8 [==============================] - 30s 4s/step - loss: 0.5809 - accuracy: 0.7350 - val_loss: 0.4981 - val_accuracy: 0.8200
Epoch 182/200
8/8 [========

In [ ]:
test_loss, test_acc = m.evaluate(ag[800:1000,...],labeldata[800:1000])


In [ ]:
labeldata[1200:1210]

In [ ]:
m.predict(ag[1200:1210])

In [ ]:
m.save('/kaggle/working/maskdetection.h5')

In [ ]:
bb = tf.models.load_model("/kaggle/working/maskdetection.h5")

In [ ]:
bb.predict(ag[1200:1210])

In [ ]:
hh = [1,2,3,4,5]
hh[:-3]